<a href="https://colab.research.google.com/github/ggmeiner22/Estimating-Camera-Pose-from-a-Planar-Object/blob/main/EstimatingCameraPoseFromAPlanarObject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goal

Given a calibrated camera (i.e., known intrinsics **K**) and a single image of a planar object, estimate the camera **pose** (i.e., extrinsics **R**, **t**).

I will:

1. build a Gradio UI to click 2D features and manage point order,
2. estimate pose **from a homography** (explicit derivation), and
3. estimate pose using **OpenCV** functions,
4. compare the two (short notes only).

Installs dependencies

In [ ]:
%pip -q install opencv-python numpy gradio matplotlib

Imports

In [ ]:
import json, io, math
import numpy as np
import cv2 as cv
import gradio as gr
import matplotlib.pyplot as plt

from matplotlib.figure import Figure